In [1]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

In [ ]:
#os.makedirs(output_folder, exist_ok=True)

In [22]:
"HCHO" in os.path.basename(nc_file_path)

True

In [1]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Hcho  = fh.groups['PRODUCT'].variables['formaldehyde_tropospheric_vertical_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Hcho_array = np.ma.filled(Hcho, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Hcho_array[x, y]):
            Hcho_value = Hcho_array[x, y]
        else:
            Hcho_value = np.nan

        output_data.append([user_lon, user_lat, Hcho_value])


    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Hcho_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/Delhi_Hcho_all_csv"
need_csv_path = "/home/sysadm/Downloads/delhi_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of NO2 files
Hcho_file_paths = []

# Iterate through the .nc files and filter out the HCHO files
for nc_file in nc_files:
    if "HCHO" in nc_file:
        Hcho_file_path = os.path.join(input_folder, nc_file)
        Hcho_file_paths.append(Hcho_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Hcho_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_115334/3165166966.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_115334/3165166966.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_115334/3165166966.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_115334/3165166966.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_115334/3165166966.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_115334/3165166966.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_115334/3165166966.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_115334/3165166966.py:37: R

In [4]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/Delhi_Hcho_all_csv'
combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Hcho/Delhi_Combine_Hcho_csv'
need_csv_path = "/home/sysadm/Downloads/delhi_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)


# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        hcho_df = df[['Hcho_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'No2' dataframe
        if combined_df is None:
            combined_df = hcho_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, hcho_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [5]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Hcho/Delhi_Combine_Hcho_csv"

final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/Delhi_2023_sep_to_oct_csv"

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Hcho_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Hcho_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)
        

In [2]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Hcho  = fh.groups['PRODUCT'].variables['formaldehyde_tropospheric_vertical_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Hcho_array = np.ma.filled(Hcho, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Hcho_array[x, y]):
            Hcho_value = Hcho_array[x, y]
        else:
            Hcho_value = np.nan

        output_data.append([user_lon, user_lat, Hcho_value])


    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Hcho_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/WB_Hcho_all_csv"
need_csv_path = "/home/sysadm/Downloads/inside_wb_1632pts_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of NO2 files
Hcho_file_paths = []

# Iterate through the .nc files and filter out the HCHO files
for nc_file in nc_files:
    if "HCHO" in nc_file:
        Hcho_file_path = os.path.join(input_folder, nc_file)
        Hcho_file_paths.append(Hcho_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Hcho_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWa

/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_48213/1061169562.py:37: RuntimeWa

In [3]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/WB_Hcho_all_csv'
combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Hcho/WB_Combine_Hcho_csv'
need_csv_path = "/home/sysadm/Downloads/inside_wb_1632pts_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)


# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        hcho_df = df[['Hcho_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'No2' dataframe
        if combined_df is None:
            combined_df = hcho_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, hcho_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [4]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Hcho/WB_Combine_Hcho_csv"

final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/WB_2023_sep_to_oct_csv"

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Hcho_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Hcho_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)
        

In [1]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Hcho  = fh.groups['PRODUCT'].variables['formaldehyde_tropospheric_vertical_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Hcho_array = np.ma.filled(Hcho, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Hcho_array[x, y]):
            Hcho_value = Hcho_array[x, y]
        else:
            Hcho_value = np.nan

        output_data.append([user_lon, user_lat, Hcho_value])


    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Hcho_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/Karnataka_Hcho_all_csv"
os.makedirs(csv_output_dir, exist_ok = True)

need_csv_path = "/home/sysadm/Downloads/karnataka_1650_points_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of NO2 files
Hcho_file_paths = []

# Iterate through the .nc files and filter out the HCHO files
for nc_file in nc_files:
    if "HCHO" in nc_file:
        Hcho_file_path = os.path.join(input_folder, nc_file)
        Hcho_file_paths.append(Hcho_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Hcho_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: i

/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1761488774.py:37: RuntimeWarning: i

In [4]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/Karnataka_Hcho_all_csv'
os.makedirs(input_folder, exist_ok = True)

combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Hcho/Karnataka_Combine_Hcho_csv'
os.makedirs(combine_csv_output_dir, exist_ok = True)


need_csv_path = "/home/sysadm/Downloads/karnataka_1650_points_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)


# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        hcho_df = df[['Hcho_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'No2' dataframe
        if combined_df is None:
            combined_df = hcho_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, hcho_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [5]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Hcho/Karnataka_Combine_Hcho_csv"
os.makedirs(input_folder, exist_ok = True)


final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/Karnataka_2023_sep_to_oct_csv"
os.makedirs(final_csv_output_dir, exist_ok = True)


# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Hcho_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Hcho_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)
        

In [6]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Hcho  = fh.groups['PRODUCT'].variables['formaldehyde_tropospheric_vertical_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Hcho_array = np.ma.filled(Hcho, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Hcho_array[x, y]):
            Hcho_value = Hcho_array[x, y]
        else:
            Hcho_value = np.nan

        output_data.append([user_lon, user_lat, Hcho_value])


    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Hcho_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/Gujrat_Hcho_all_csv"
os.makedirs(csv_output_dir, exist_ok = True)

need_csv_path = "/home/sysadm/Downloads/gujarat_1485_points_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of NO2 files
Hcho_file_paths = []

# Iterate through the .nc files and filter out the HCHO files
for nc_file in nc_files:
    if "HCHO" in nc_file:
        Hcho_file_path = os.path.join(input_folder, nc_file)
        Hcho_file_paths.append(Hcho_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Hcho_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid v

/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/928923278.py:37: RuntimeWarning: invalid v

In [7]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/Gujrat_Hcho_all_csv'
os.makedirs(input_folder, exist_ok = True)

combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Hcho/Gujrat_Combine_Hcho_csv'
os.makedirs(combine_csv_output_dir, exist_ok = True)


need_csv_path = "/home/sysadm/Downloads/gujarat_1485_points_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)


# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        hcho_df = df[['Hcho_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'No2' dataframe
        if combined_df is None:
            combined_df = hcho_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, hcho_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [8]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Hcho/Gujrat_Combine_Hcho_csv"
os.makedirs(input_folder, exist_ok = True)


final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/Gujrat_2023_sep_to_oct_csv"
os.makedirs(final_csv_output_dir, exist_ok = True)


# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Hcho_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Hcho_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)
        

In [13]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Hcho  = fh.groups['PRODUCT'].variables['formaldehyde_tropospheric_vertical_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Hcho_array = np.ma.filled(Hcho, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Hcho_array[x, y]):
            Hcho_value = Hcho_array[x, y]
        else:
            Hcho_value = np.nan

        output_data.append([user_lon, user_lat, Hcho_value])


    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Hcho_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/Orissa_Hcho_all_csv"
os.makedirs(csv_output_dir, exist_ok = True)

need_csv_path = "/home/sysadm/Downloads/orissa_1540_points_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of NO2 files
Hcho_file_paths = []

# Iterate through the .nc files and filter out the HCHO files
for nc_file in nc_files:
    if "HCHO" in nc_file:
        Hcho_file_path = os.path.join(input_folder, nc_file)
        Hcho_file_paths.append(Hcho_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Hcho_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid v

/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/720947034.py:37: RuntimeWarning: invalid v

In [14]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/Orissa_Hcho_all_csv'
os.makedirs(input_folder, exist_ok = True)

combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Hcho/Orissa_Combine_Hcho_csv'
os.makedirs(combine_csv_output_dir, exist_ok = True)


need_csv_path = "/home/sysadm/Downloads/orissa_1540_points_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)


# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        hcho_df = df[['Hcho_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'No2' dataframe
        if combined_df is None:
            combined_df = hcho_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, hcho_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [15]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Hcho/Orissa_Combine_Hcho_csv"
os.makedirs(input_folder, exist_ok = True)


final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/Orissa_2023_sep_to_oct_csv"
os.makedirs(final_csv_output_dir, exist_ok = True)


# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Hcho_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Hcho_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)
        

In [16]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Hcho  = fh.groups['PRODUCT'].variables['formaldehyde_tropospheric_vertical_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Hcho_array = np.ma.filled(Hcho, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Hcho_array[x, y]):
            Hcho_value = Hcho_array[x, y]
        else:
            Hcho_value = np.nan

        output_data.append([user_lon, user_lat, Hcho_value])


    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Hcho_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/Tamilnadu_Hcho_all_csv"
os.makedirs(csv_output_dir, exist_ok = True)

need_csv_path = "/home/sysadm/Downloads/tamilnadu_1400_points_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of NO2 files
Hcho_file_paths = []

# Iterate through the .nc files and filter out the HCHO files
for nc_file in nc_files:
    if "HCHO" in nc_file:
        Hcho_file_path = os.path.join(input_folder, nc_file)
        Hcho_file_paths.append(Hcho_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Hcho_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: i

/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/1780048166.py:37: RuntimeWarning: i

In [17]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/Tamilnadu_Hcho_all_csv'
os.makedirs(input_folder, exist_ok = True)

combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Hcho/Tamilnadu_Combine_Hcho_csv'
os.makedirs(combine_csv_output_dir, exist_ok = True)


need_csv_path = "/home/sysadm/Downloads/tamilnadu_1400_points_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)


# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        hcho_df = df[['Hcho_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'No2' dataframe
        if combined_df is None:
            combined_df = hcho_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, hcho_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [18]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Hcho/Tamilnadu_Combine_Hcho_csv"
os.makedirs(input_folder, exist_ok = True)


final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/Tamilnadu_2023_sep_to_oct_csv"
os.makedirs(final_csv_output_dir, exist_ok = True)


# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Hcho_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Hcho_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)
        

In [19]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Hcho  = fh.groups['PRODUCT'].variables['formaldehyde_tropospheric_vertical_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Hcho_array = np.ma.filled(Hcho, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Hcho_array[x, y]):
            Hcho_value = Hcho_array[x, y]
        else:
            Hcho_value = np.nan

        output_data.append([user_lon, user_lat, Hcho_value])


    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Hcho_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/Maharashtra_Hcho_all_csv"
os.makedirs(csv_output_dir, exist_ok = True)

need_csv_path = "/home/sysadm/Downloads/maharashtra_1696_points_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of NO2 files
Hcho_file_paths = []

# Iterate through the .nc files and filter out the HCHO files
for nc_file in nc_files:
    if "HCHO" in nc_file:
        Hcho_file_path = os.path.join(input_folder, nc_file)
        Hcho_file_paths.append(Hcho_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Hcho_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: i

/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_6323/3730754377.py:37: RuntimeWarning: i

In [20]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Hcho/Maharashtra_Hcho_all_csv'
os.makedirs(input_folder, exist_ok = True)

combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Hcho/Maharashtra_Combine_Hcho_csv'
os.makedirs(combine_csv_output_dir, exist_ok = True)


need_csv_path = "/home/sysadm/Downloads/maharashtra_1696_points_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)


# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        hcho_df = df[['Hcho_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'No2' dataframe
        if combined_df is None:
            combined_df = hcho_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, hcho_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [2]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Hcho/Maharashtra_Combine_Hcho_csv"
os.makedirs(input_folder, exist_ok = True)


final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Hcho/Maharashtra_2023_sep_to_oct_csv"
os.makedirs(final_csv_output_dir, exist_ok = True)


# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Hcho_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Hcho_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)
        